# Week 06 Live Coding Demo: Data I/O & Visualization

## Demo Flow
**Text I/O** → **Binary I/O + Performance Analysis** → **Physics Visualizations** → **2D Animation**

Each section builds on the previous one, showing you how to handle real physics data and create compelling visualizations that help you understand physical phenomena.

## 1. Text-based I/O — CSV (pandas), JSON metadata, TXT (NumPy)

**When to use text formats:**
- **CSV**: Tabular data that needs to be shared with others or opened in Excel
- **JSON**: Metadata, configuration files, or structured data that needs to be human-readable
- **TXT**: Simple numerical arrays, especially when you need to inspect the data manually

**Key advantages:**
- Human-readable and editable
- Platform-independent
- Can be opened in any text editor
- Good for sharing data with collaborators

**Key disadvantages:**
- Larger file sizes
- Slower read/write speeds
- Potential precision loss for floating-point numbers

In [ ]:
# Import essential libraries for data manipulation
import numpy as np, pandas as pd

# === CSV Demo: I-V Characteristic of a Resistor ===
# This simulates measuring current vs voltage for a 120Ω resistor
# with some measurement noise (typical of real lab equipment)

# Generate voltage data: sweep from -1V to +1V in 41 steps
V_V = np.linspace(-1.0, 1.0, 41)  # Voltage in Volts

# Resistor value and random number generator for reproducible "noise"
R_ohm = 120.0  # Resistance in Ohms
rng = np.random.default_rng(42)  # Fixed seed for reproducible results

# Calculate current using Ohm's law: I = V/R, plus measurement noise
I_A = V_V / R_ohm + rng.normal(0, 0.0005, size=V_V.size)  # Current in Amperes

# Create a pandas DataFrame - this is the standard way to handle tabular data
df = pd.DataFrame({"V_V": V_V, "I_A": I_A})

# Save to CSV file
csv_path = "wk06_iv_curve.csv"
df.to_csv(csv_path, index=False)  # index=False prevents saving row numbers

# Read the CSV back and verify the data
iv = pd.read_csv(csv_path, dtype={"V_V":"float64", "I_A":"float64"})
print("Head:\n", iv.head(5).to_string(index=False))
print("\nDtypes:\n", iv.dtypes.to_string())


In [ ]:
# === JSON Demo: Storing Metadata ===
# JSON is perfect for storing experiment metadata, configuration settings,
# and any structured information that needs to be human-readable

import json

# Create metadata dictionary - this contains all the important information
# about our experiment that we want to save alongside the data
meta = {
    "experiment": "IV_sweep",           # Experiment type
    "R_model_ohm": 120.0,              # Expected resistance value
    "temperature_C": 21.5,             # Lab temperature (affects resistance)
    "author": "PHYS77-demo",           # Who did the experiment
    "units": {"V_V": "V", "I_A": "A"}  # Units for each column in our data
}

# Save metadata to JSON file
json_path = "wk06_iv_meta.json"
with open(json_path, "w") as f:
    json.dump(meta, f, indent=2)  # indent=2 makes it pretty-printed

# Read the JSON back to verify
with open(json_path) as f:
    meta2 = json.load(f)

print("Loaded meta keys:", sorted(meta2.keys()))
print("Units mapping:", meta2["units"])

# JSON is great because:
# 1. It's human-readable (you can open it in any text editor)
# 2. It's structured (nested dictionaries and lists)
# 3. It's widely supported (most programming languages can read JSON)
# 4. It's perfect for configuration files and metadata


In [ ]:
# === TXT Demo: Simple Numerical Arrays ===
# NumPy's savetxt/loadtxt is perfect for simple 2D arrays of numbers
# This is often the fastest way to save/load pure numerical data

import numpy as np

# Generate a damped oscillation: x(t) = e^(-3t) * sin(2π*6*t)
# This represents a physical system like a damped harmonic oscillator
t_s = np.linspace(0, 1.0, 21)  # Time array: 0 to 1 second, 21 points
x = np.exp(-3*t_s) * np.sin(2*np.pi*6*t_s)  # Damped sine wave

# Stack time and position into a 2D array (columns: time, position)
arr = np.column_stack([t_s, x])

# Save to text file with header
txt_path = "wk06_tx_series.txt"
np.savetxt(txt_path, arr, 
           header="t_s x_au",    # Column headers
           comments="",          # No comment character (default is '#')
           fmt="%.6f")          # Format: 6 decimal places

# Load the data back
loaded = np.loadtxt(txt_path, skiprows=1)  # Skip the header row
print("TXT shape:", loaded.shape)
print("First 3 rows:\n", loaded[:3])
print("Header preview:", open(txt_path).read().splitlines()[0])

# When to use np.savetxt/loadtxt:
# ✓ Simple 2D numerical arrays
# ✓ When you need to inspect data manually
# ✓ Quick and dirty data export
# ✗ Not good for complex data structures
# ✗ Slower than binary formats for large arrays


## 2. Binary I/O (.npy/.npz) — Performance & Size vs TXT

**When to use binary formats:**
- **.npy**: Single NumPy arrays that you need to save/load quickly
- **.npz**: Multiple arrays that belong together (like different variables from the same experiment)

**Key advantages:**
- **Much faster** read/write speeds (especially for large arrays)
- **Smaller file sizes** (no text formatting overhead)
- **Exact precision** preservation (no rounding errors from text conversion)
- **Preserves data types** (int32, float64, etc.)

**Key disadvantages:**
- Not human-readable (you can't open in a text editor)
- NumPy-specific (though other languages can read .npy files)
- Less portable than text formats

**Rule of thumb:** Use binary for internal data processing, text for sharing with others.

In [ ]:
# === Performance Comparison: Binary vs Text I/O ===
# This demo shows the dramatic performance difference between binary and text formats
# We'll create a large dataset and time the save/load operations

import numpy as np, os, time

# Create a large dataset: 200,000 points of sine wave data
N = 200_000  # Number of data points
a32 = np.sin(np.linspace(0, 200*np.pi, N, dtype=np.float32))  # Sine wave, 32-bit precision
A2 = np.vstack([a32, a32**2, a32**3]).T  # Stack: sin(x), sin²(x), sin³(x) as columns

# === Test 1: .npy format (single array) ===
print("Testing .npy format...")
t0 = time.perf_counter()
np.save("wk06_A2.npy", A2)  # Save single array
save_npy_time = time.perf_counter() - t0

t1 = time.perf_counter()
A2_loaded = np.load("wk06_A2.npy")  # Load single array
load_npy_time = time.perf_counter() - t1
npy_size = os.path.getsize("wk06_A2.npy")

# === Test 2: .npz format (multiple arrays) ===
print("Testing .npz format...")
t2 = time.perf_counter()
np.savez("wk06_three_arrays.npz", sin=a32, sin2=a32**2, sin3=a32**3)  # Save multiple named arrays
npz_time = time.perf_counter() - t2
npz_size = os.path.getsize("wk06_three_arrays.npz")

# === Test 3: .txt format (for comparison) ===
print("Testing .txt format...")
t3 = time.perf_counter()
np.savetxt("wk06_A2.txt", A2, fmt="%.6e")  # Save as text with scientific notation
save_txt_time = time.perf_counter() - t3
txt_size = os.path.getsize("wk06_A2.txt")

t4 = time.perf_counter()
A2_txt = np.loadtxt("wk06_A2.txt")  # Load text file
load_txt_time = time.perf_counter() - t4

# === Results Summary ===
print("\n" + "="*60)
print("PERFORMANCE COMPARISON RESULTS:")
print("="*60)
print(f".npy save: {save_npy_time:.3f} s, load: {load_npy_time:.3f} s, size: {npy_size/1e6:.2f} MB")
print(f".npz (3 arrays) save: {npz_time:.3f} s, size: {npz_size/1e6:.2f} MB")
print(f".txt save: {save_txt_time:.3f} s, load: {load_txt_time:.3f} s, size: {txt_size/1e6:.2f} MB")

# Verify data integrity
print("\nData integrity checks:")
print("Binary data matches original:", np.allclose(A2_loaded, A2))
print("Text data matches original:", np.allclose(A2_txt, A2.astype(np.float64)))

# Calculate speedup ratios
save_speedup = save_txt_time / save_npy_time
load_speedup = load_txt_time / load_npy_time
size_ratio = txt_size / npy_size

print(f"\nBinary vs Text Performance:")
print(f"Save speedup: {save_speedup:.1f}x faster")
print(f"Load speedup: {load_speedup:.1f}x faster") 
print(f"Size reduction: {size_ratio:.1f}x smaller")

# Key takeaway: For large datasets, binary formats are dramatically faster and smaller!


## 3. Visualization Demos (Physics-Motivated)


### Demo 1 — 1D Line & Error Bars: RC Charging Curve with Noise

**Physics Background:**
An RC circuit consists of a resistor (R) and capacitor (C) in series. When a voltage is applied, the capacitor charges up exponentially:
- **Charging equation**: V(t) = V₀(1 - e^(-t/τ)) where τ = RC is the time constant
- **Physical meaning**: The capacitor voltage approaches the source voltage V₀ exponentially
- **Time constant τ**: Time for voltage to reach ~63% of final value

**Why error bars matter:**
Real measurements always have uncertainty due to:
- Instrument noise and resolution limits
- Environmental factors (temperature, electromagnetic interference)
- Systematic errors in the measurement setup

**Plotting techniques we'll learn:**
- `errorbar()` for showing measurement uncertainty
- Theory vs experiment comparison
- Proper axis labeling with units
- Grid and legend for clarity

In [ ]:
# === RC Circuit Charging Curve with Error Bars ===
import numpy as np, matplotlib.pyplot as plt

# === Set up the RC circuit parameters ===
t = np.linspace(0, 5, 60)  # Time array: 0 to 5 seconds, 60 points
V0, R, C = 5.0, 10e3, 100e-6  # Source voltage, resistance, capacitance
# R = 10 kΩ, C = 100 μF → time constant τ = RC = 1 second

# === Calculate the theoretical charging curve ===
# V(t) = V₀(1 - e^(-t/τ)) where τ = RC
V_true = V0 * (1 - np.exp(-t/(R*C)))

# === Simulate realistic measurement data with noise ===
rng = np.random.default_rng(3)  # Fixed seed for reproducible results
sigma = 0.07  # Standard deviation of measurement noise (70 mV)
V_meas = V_true + rng.normal(0, sigma, size=t.size)  # Add Gaussian noise

# === Create the plot ===
plt.figure(figsize=(6,3.5))

# Plot measured data with error bars
plt.errorbar(t, V_meas, yerr=sigma, 
             fmt="o",           # Circle markers
             capsize=3,         # Error bar cap size
             label="measured",  # Legend label
             markersize=4)      # Marker size

# Plot theoretical curve
plt.plot(t, V_true, lw=2, label="theory", color="red")

# === Formatting and labels ===
plt.xlabel("time t [s]")        # Always include units!
plt.ylabel("voltage V [V]")     # Always include units!
plt.title("RC Charging")        # Descriptive title
plt.grid(True, alpha=0.3)       # Light grid for readability
plt.legend()                    # Show legend
plt.tight_layout()              # Prevent label cutoff
plt.show()


### Demo 2 — Scatter with Color Mapping: Projectile Range vs Angle & Speed

**Physics Background:**
Projectile motion under gravity follows these equations:
- **Range formula**: R = (v₀² sin(2θ))/g where v₀ is initial speed, θ is launch angle
- **Key insight**: Maximum range occurs at θ = 45° (when sin(2θ) = 1)
- **Dependence**: Range depends on BOTH speed and angle

**Why color mapping is powerful:**
- Shows 3 variables simultaneously: x (angle), y (speed), color (range)
- Reveals patterns that would be invisible in 2D plots
- Makes it easy to identify optimal launch conditions

**Plotting techniques we'll learn:**
- `scatter()` with color mapping using the `c` parameter
- Colormap selection (`cmap`) for different data types
- Colorbar (`colorbar()`) to show the color-to-value mapping
- Edge colors and transparency for better visibility

In [ ]:
# === Projectile Motion: Range vs Launch Angle and Speed ===
import numpy as np, matplotlib.pyplot as plt

# === Physics constants and parameters ===
g = 9.81  # Gravitational acceleration [m/s²]

# === Generate random launch conditions ===
rng = np.random.default_rng(0)  # Fixed seed for reproducible results
angles_deg = rng.uniform(10, 80, 400)  # Launch angles: 10° to 80°
speeds = rng.uniform(5, 30, 400)       # Initial speeds: 5 to 30 m/s
angles = np.deg2rad(angles_deg)        # Convert to radians for calculations

# === Calculate projectile range ===
# Range formula: R = (v₀² sin(2θ))/g
# This is the horizontal distance traveled before hitting the ground
R = speeds**2 * np.sin(2*angles) / g

# === Create the scatter plot with color mapping ===
plt.figure(figsize=(6,3.5))

# Scatter plot: x=angle, y=speed, color=range
sc = plt.scatter(angles_deg, speeds, 
                 c=R,                    # Color represents range
                 cmap="viridis",         # Colormap: dark=low, bright=high
                 s=30,                   # Marker size
                 edgecolor="none",       # No edge color for cleaner look
                 alpha=0.8)              # Slight transparency

# === Add colorbar and labels ===
plt.xlabel("launch angle [deg]")        # Always include units!
plt.ylabel("speed [m/s]")               # Always include units!
cbar = plt.colorbar(sc)                 # Add colorbar
cbar.set_label("range [m] (ideal)")     # Label the colorbar with units
plt.title("Projectile range landscape") # Descriptive title
plt.tight_layout()                      # Prevent label cutoff
plt.show()

# === Physics insights from this plot ===
# 1. Maximum range occurs around 45° (as expected from theory)
# 2. Higher speeds generally give longer ranges
# 3. The color gradient shows the "landscape" of possible ranges
# 4. You can easily identify optimal launch conditions (bright yellow regions)

# === Why "viridis" colormap? ===
# - Perceptually uniform (equal steps in color = equal steps in data)
# - Colorblind-friendly
# - Works well in both color and grayscale
# - Default choice for scientific visualization


### Demo 3 — Electrostatic Potential of Two Point Charges (Heatmap + Contours + Colormap Survey)

**Physics Background:**
This demo visualizes the electric field of an electric dipole:
- **Dipole**: Two equal and opposite charges (+q and -q) separated by distance d
- **Potential**: V(r) = kq₁/r₁ + kq₂/r₂ where r₁, r₂ are distances to each charge
- **Electric field**: E = -∇V (negative gradient of potential)
- **Field lines**: Curves tangent to the electric field direction

**Why this visualization is powerful:**
- **Heatmaps** show the potential landscape (like a topographic map)
- **Contour lines** show equipotential surfaces (where V = constant)
- **Streamlines** show the direction of electric field lines
- **Multiple colormaps** demonstrate how color choice affects interpretation

**Advanced plotting techniques:**
- `imshow()` for 2D heatmaps
- `contour()` and `clabel()` for equipotential lines
- `streamplot()` for field line visualization
- `np.gradient()` for calculating electric field from potential
- Masking to avoid singularities near point charges

In [ ]:
# === Electric Dipole: Potential and Field Visualization ===
import numpy as np, matplotlib.pyplot as plt

# === Set up the dipole geometry and coordinate grid ===
d = 1.0                                         # Half-separation between charges
q1, q2 = +1.0, -1.0                             # Positive and negative charges
x = np.linspace(-3, 3, 300)                     # x-coordinates: -3 to +3
y = np.linspace(-2, 2, 200)                     # y-coordinates: -2 to +2
X, Y = np.meshgrid(x, y, indexing="xy")         # Create 2D coordinate grids

# === Calculate electrostatic potential ===
# For point charges: V = kq/r (we set k=1 for simplicity)
r1 = np.hypot(X + d, Y)                         # Distance to +q charge at (-d, 0)
r2 = np.hypot(X - d, Y)                         # Distance to -q charge at (+d, 0)
eps = 1e-3                                      # Small cutoff to avoid 1/0 singularity
V = q1/np.maximum(r1, eps) + q2/np.maximum(r2, eps)  # Total potential

# === Create two-panel figure ===
fig, ax = plt.subplots(1, 2, figsize=(10, 3.6))

# === LEFT PANEL: Potential with equipotential contours ===
# Heatmap of potential
im = ax[0].imshow(V, extent=[x.min(), x.max(), y.min(), y.max()], origin="lower",
                  cmap="seismic", vmin=-3, vmax=3, aspect="auto")
# "seismic" colormap: red=positive, white=zero, blue=negative

# Add equipotential contour lines
cs = ax[0].contour(X, Y, V, levels=np.linspace(-3, 3, 13), 
                   colors="k", linewidths=0.6)
ax[0].clabel(cs, inline=True, fontsize=7)       # Label contour lines

# Mark the charges
ax[0].scatter([-d, d], [0, 0], c=["red", "blue"], s=40, zorder=3)
ax[0].set_title("Dipole potential V(x,y)")
ax[0].set_xlabel("x"); ax[0].set_ylabel("y")
cb = fig.colorbar(im, ax=ax[0], fraction=0.05, pad=0.04)
cb.set_label("V [a.u.]")

# === RIGHT PANEL: Electric field magnitude and streamlines ===
# Calculate electric field from potential: E = -∇V
dx, dy = x[1]-x[0], y[1]-y[0]                   # Grid spacing
dVdy, dVdx = np.gradient(V, dy, dx)             # Gradient components
Ex, Ey = -dVdx, -dVdy                           # Electric field components
E = np.hypot(Ex, Ey)                            # Field magnitude |E|

# Mask regions near charges to avoid singularities
mask = (r1 < 0.15) | (r2 < 0.15)                # Near-charge regions
Exm = np.ma.masked_where(mask, Ex)              # Masked field components
Eym = np.ma.masked_where(mask, Ey)
Em  = np.ma.masked_where(mask, E)

# Use robust scaling for field magnitude
valid = ~mask
vmaxE = np.percentile(E[valid], 95) if np.any(valid) else E.max()

# Heatmap of field magnitude
im1 = ax[1].imshow(Em, extent=[x.min(), x.max(), y.min(), y.max()], origin="lower",
                   cmap="inferno", vmin=0, vmax=vmaxE, aspect="auto")
# "inferno" colormap: black=low, bright=high (good for magnitude)

# Streamlines show field direction
ax[1].streamplot(X, Y, Exm, Eym, density=1.2, linewidth=0.6, 
                 color=(1, 1, 1, 0.5), arrowsize=0.8)

# Mark the charges
ax[1].scatter([-d, d], [0, 0], c=["red", "blue"], s=40, zorder=3)
ax[1].set_title("|E| and field lines")
ax[1].set_xlabel("x"); ax[1].set_ylabel("y")
cb1 = fig.colorbar(im1, ax=ax[1], fraction=0.05, pad=0.04)
cb1.set_label("|E| [a.u.]")

# === COLORMAP SURVEY: How color choice affects interpretation ===
cmaps = ["seismic", "coolwarm", "twilight", "viridis", "cividis", "magma", "plasma", "Greys"]
n = len(cmaps); cols = 4; rows = (n + cols - 1) // cols
fig2, axes = plt.subplots(rows, cols, figsize=(10, 2.0*rows), constrained_layout=True)

for i, name in enumerate(cmaps):
    r, c = divmod(i, cols)
    axc = axes[r, c] if rows > 1 else axes[c]
    axc.imshow(V, origin="lower", extent=[x.min(), x.max(), y.min(), y.max()],
               cmap=name, vmin=-3, vmax=3, aspect="auto")
    axc.scatter([-d, d], [0, 0], c=["red", "blue"], s=15)
    axc.set_title(name, fontsize=9)
    axc.set_xticks([]); axc.set_yticks([])      # Remove tick marks for cleaner look

# Hide unused subplot cells
for j in range(n, rows*cols):
    r, c = divmod(j, cols)
    (axes[r, c] if rows > 1 else axes[c]).axis("off")

plt.show()

# === Key Physics Insights ===
# 1. Potential is positive near +q, negative near -q
# 2. Equipotential lines are perpendicular to field lines
# 3. Field is strongest near the charges (bright regions in right panel)
# 4. Field lines start at +q and end at -q
# 5. Different colormaps can emphasize different features of the same data

### Demo 4 — 3D Surface: 2D Quantum Harmonic Oscillator (Lz=+ħ)

**Physics Background:**
This demo visualizes a quantum state with definite angular momentum:
- **2D Harmonic Oscillator**: Particle in a 2D parabolic potential V(x,y) = ½mω²(x²+y²)
- **Angular momentum eigenstate**: Lz = +ħ (definite angular momentum about z-axis)
- **Wavefunction construction**: ψ = (ψ₁₀ + iψ₀₁)/√2 where ψ₁₀, ψ₀₁ are 1D oscillator states
- **Physical meaning**: The particle has a "whirling" motion with definite angular momentum

**Why 3D visualization is powerful:**
- **Height** represents probability density |ψ|² (where particle is likely to be found)
- **Color** represents phase arg(ψ) (quantum mechanical phase information)
- **Combined view** shows both the "where" and "how" of quantum probability

**Advanced 3D plotting techniques:**
- `plot_surface()` with custom face colors
- Phase-to-color mapping using cyclic colormaps
- 3D axis labeling and view control
- Colorbar for phase interpretation


In [ ]:
# === 2D Quantum Harmonic Oscillator with Angular Momentum ===
import numpy as np, matplotlib.pyplot as plt
from matplotlib import cm

# === Define 1D harmonic oscillator eigenfunctions ===
# For a 1D harmonic oscillator with ω=1 and m=1, the eigenfunctions are:
# ψ_n(x) = (1/√(2^n n! √π)) * H_n(x) * e^(-x²/2)
# where H_n(x) are Hermite polynomials

def H0(x): 
    """Hermite polynomial H₀(x) = 1"""
    return np.ones_like(x)

def H1(x): 
    """Hermite polynomial H₁(x) = 2x"""
    return 2*x

def psi_n(x, n):
    """Normalized 1D harmonic oscillator eigenfunction ψ_n(x)"""
    if n == 0: 
        # Ground state: ψ₀(x) = (1/√π)^(1/4) * e^(-x²/2)
        return (np.pi**-0.25) * H0(x) * np.exp(-x**2/2)
    if n == 1: 
        # First excited state: ψ₁(x) = (1/√π)^(1/4) * (1/√2) * 2x * e^(-x²/2)
        return (np.pi**-0.25) / np.sqrt(2) * H1(x) * np.exp(-x**2/2)
    raise ValueError("This demo only uses n ∈ {0,1}")

# === Set up coordinate grid ===
x = np.linspace(-3, 3, 160)                      # x-coordinates
y = np.linspace(-3, 3, 160)                      # y-coordinates  
X, Y = np.meshgrid(x, y, indexing="xy")          # 2D coordinate grids

# === Construct 2D angular momentum eigenstate ===
# For a 2D harmonic oscillator, we can build states with definite angular momentum
# by combining the 1D eigenfunctions. The state with Lz = +ħ is:
# ψ = (ψ₁₀ + iψ₀₁)/√2
# where ψ₁₀ = ψ₁(x)ψ₀(y) and ψ₀₁ = ψ₀(x)ψ₁(y)

psi10 = psi_n(X, 1) * psi_n(Y, 0)                # ψ₁₀(x,y) = ψ₁(x)ψ₀(y)
psi01 = psi_n(X, 0) * psi_n(Y, 1)                # ψ₀₁(x,y) = ψ₀(x)ψ₁(y)
psi = (psi10 + 1j*psi01)/np.sqrt(2)              # Angular momentum eigenstate

# === Extract probability density and phase ===
P = np.abs(psi)**2                               # Probability density |ψ|²
phase = np.angle(psi)                            # Phase arg(ψ) ∈ [-π, π]

# === Set up color mapping for phase ===
# Use a cyclic colormap since phase wraps around from -π to +π
cmap = cm.get_cmap("twilight")                   # Cyclic colormap: dark=0, bright=±π
norm = plt.Normalize(-np.pi, np.pi)              # Normalize phase to [-π, π]
colors = cmap(norm(phase))                       # Convert phase to RGBA colors

# === Create 3D surface plot ===
fig = plt.figure(figsize=(6.5, 4.6))
ax = fig.add_subplot(111, projection="3d")

# Plot surface: height = probability density, color = phase
ax.plot_surface(X, Y, P, 
                facecolors=colors,               # Custom colors for each face
                linewidth=0,                     # No edge lines
                antialiased=True,                # Smooth rendering
                shade=False)                     # No shading (we use color instead)

# === Formatting and labels ===
ax.set_xlabel("x")
ax.set_ylabel("y") 
ax.set_zlabel("|ψ|²")
ax.set_title("2D HO (Lz=+ħ): density (height) and phase (color)")

# Set viewing angle for best visibility
ax.view_init(elev=75, azim=-60)                  # 75° elevation, -60° azimuth

# === Add colorbar for phase ===
# Create a dummy mappable for the colorbar
mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
mappable.set_array([])                           # Empty array (we just need the colormap)
cb = fig.colorbar(mappable, ax=ax, fraction=0.05, pad=0.08)
cb.set_label("phase arg(ψ) [rad]")

plt.tight_layout()
plt.show()

# === Physics Interpretation ===
# 1. The height shows where the particle is most likely to be found
# 2. The color shows the quantum mechanical phase (like a "wave pattern")
# 3. The phase varies smoothly, creating a "whirling" pattern
# 4. This corresponds to the particle having definite angular momentum Lz = +ħ
# 5. The probability density has a characteristic "dumbbell" shape


### Demo 5 — 2D Animation: Interference of Two Coherent Point Sources

**Physics Background:**
This demo shows wave interference - a fundamental phenomenon in physics:
- **Two coherent sources**: Both sources oscillate with the same frequency and phase
- **Wave equation**: Each source creates circular waves that propagate outward
- **Interference**: Waves from both sources add together (constructive/destructive interference)
- **Result**: Interference pattern with bright and dark regions

**Why animation is powerful:**
- Shows the **time evolution** of the wave pattern
- Reveals how interference patterns form and change
- Demonstrates the wave nature of the phenomenon
- Makes abstract concepts concrete and visual

**Animation techniques we'll learn:**
- `FuncAnimation()` for creating smooth animations
- `PillowWriter` for saving as GIF files
- `set_data()` for updating plot data efficiently
- `blit=True` for smooth animation performance

In [ ]:
# === Wave Interference Animation: Two Coherent Point Sources ===
import numpy as np, matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image, display

# === Set up wave parameters ===
lam = 0.5                                         # Wavelength
k = 2*np.pi/lam                                   # Wave number k = 2π/λ
omega = 2*np.pi*1                                 # Angular frequency (1 Hz)
d = 1.0                                           # Separation between sources

# === Create coordinate grid ===
x = np.linspace(-3, 3, 200)                      # x-coordinates
y = np.linspace(-2, 2, 150)                      # y-coordinates
X, Y = np.meshgrid(x, y, indexing="xy")          # 2D coordinate grids

# === Calculate distances from each source ===
# Source 1 at (-d/2, 0), Source 2 at (+d/2, 0)
r1 = np.hypot(X + d/2, Y)                        # Distance to source 1
r2 = np.hypot(X - d/2, Y)                        # Distance to source 2

# === Set up the plot ===
fig, ax = plt.subplots(figsize=(6,3.6))

# Create initial image (will be updated in animation)
im = ax.imshow(np.zeros_like(X), 
               extent=[x.min(),x.max(),y.min(),y.max()], 
               origin="lower", 
               cmap="viridis",                    # Colormap for wave amplitude
               vmin=-2, vmax=2,                   # Color scale limits
               animated=True)                     # Enable animation optimization

# Mark the two sources
ax.scatter([-d/2, d/2], [0, 0], c=["white","white"], s=20, marker="x")
ax.set_title("Interference of Two Coherent Sources")
ax.set_xlabel("x"); ax.set_ylabel("y")

# === Animation function ===
def frame(i):
    """Generate frame i of the animation"""
    t = i * 0.05                                  # Time for this frame
    
    # Calculate wave field at time t
    # Each source creates a wave: sin(kr - ωt)
    # Total field is the sum of both waves (superposition principle)
    field = np.sin(k*r1 - omega*t) + np.sin(k*r2 - omega*t)
    
    # Update the image data
    im.set_data(field)
    return (im,)                                  # Return tuple for blit optimization

# === Create and save animation ===
ani = FuncAnimation(fig, frame, 
                    frames=60,                    # 60 frames total
                    interval=60,                  # 60 ms between frames
                    blit=True)                    # Use blitting for smooth animation

# Save as GIF file
gif_path = "wk06_interference.gif"
ani.save(gif_path, writer=PillowWriter(fps=20))  # 20 frames per second
plt.close(fig)                                    # Close figure to free memory

print("Saved", gif_path)
display(Image(filename=gif_path))

# === Physics Explanation ===
# 1. Each source creates circular waves that propagate outward
# 2. Where waves meet in phase (crest+crest), we get constructive interference (bright)
# 3. Where waves meet out of phase (crest+trough), we get destructive interference (dark)
# 4. The interference pattern creates alternating bright and dark bands
# 5. The pattern moves and changes as the waves propagate

# === Animation Tips ===
# - Use blit=True for smooth animations (only redraws changed parts)
# - Keep frame count reasonable (60 frames = 3 seconds at 20 fps)
# - Use appropriate colormap (viridis works well for wave amplitudes)
# - Save as GIF for easy sharing, or MP4 for higher quality
